### Assignment 5.1: thermal wind balance


In this assignment you will analyze CTD measurements near Japan and calculate the vertical shear assuming thermal wind balance. 

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gsw
import cmocean

First we will have to read the _csv_ files. Take a look at the comments and header of the first CTD dataset by running the following cell.

In [ ]:
header = pd.read_csv('Japan/CTD_Japan_station_1.csv', nrows=5,  header=None)
header

To load the data in _xarray's_ useful format, we first create a new dataset with empty data. The dimensions _trajectory_ and _obs_ represent the length of each measurement and the number of measurements, respectively.

In [ ]:
ctd = xr.Dataset(
    data_vars={
        'P':(('trajectory','obs'),np.zeros((399,5)),{'long_name':'pressure','units':'hPa'}),
        'T':(('trajectory','obs'),np.zeros((399,5)),{'long_name':'temperature','units':'degC'}),
        'S':(('trajectory','obs'),np.zeros((399,5)),{'long_name':'salinity','units':'g kg-1'}),
    },
    coords={
        'trajectory':('trajectory',range(399)),
        'obs':('obs',range(1,6)),
        'lat':('obs',np.zeros((5)),{'long_name':'latitude','units':'degrees_north'}),
        'lon':('obs',np.zeros((5)),{'long_name':'longitude','units':'degrees_east'}),
        'start_time':('obs',np.zeros((5), dtype='<M8[ns]'),{'long_name':'start_time'}),
        'end_time':('obs',np.zeros((5), dtype='<M8[ns]'),{'long_name':'end_time'}),
    }
)
ctd

__a)__ Fill the dataset with values from the csv files.

__b)__ Integrate the hydrostatic balance equation to compute the depth z (negative everywhere) for all measurements. Use the TEOS10 approximation for density and add depth and density as variables to the dataset. Plot z for one of the CTD casts.

As you can see, the CTD sinks and rises with a constant velocity.  

__c)__ Select the data during ascent **and** interpolate to depth levels every five metres. For the latter you can use `xr.apply_ufunc`, offering a way to apply functions on a dataset that may modify one or more specific dimensions, like e.g. `np.interp`. You only have to fill in your code in the `interp_1d` function, that is being applied along the `trajectory` dimension. This function interpolates a one-dimensional array `fp`, from its (core) dimension `trajectory` to the new dimension `zlev`. If you want to learn more about `xr.apply_ufunc`, check out this [example](https://docs.xarray.dev/en/stable/examples/apply_ufunc_vectorize_1d.html), which is very related to this exercise.

In [ ]:
#ctd_up =  # upward portion of dataset

def interp_1d(fp, xp, x):
    """interpolate fp(xp) to x

    args:
    fp : array(k)
        variable to interpolate
    xp : array(k)
        coordinates of fp
    x : array(k)
        new coordinates of interpolated variable

    returns:
    array(k)
        values of fp interpolated to x
    """
    return

In [ ]:
# you do not have to change this
zlev = np.arange(-5,-2000,-10) # new depth levels
ctd_interp = xr.apply_ufunc(
    interp_1d, # function to apply along core dimension
    ctd_up,    # arg1
    ctd_up.z,  # arg2
    zlev,      # arg3
    input_core_dims=[['trajectory'],['trajectory'],['zlev']], # core dimension per arg
    output_core_dims=[['zlev']],      # core dimension output
    exclude_dims=set(['trajectory']), # input dimension that may disappear in output
    vectorize=True,   # vectorizes args so interp_1d receives one-dimensional data
    keep_attrs=True,  # keep array attributes
    on_missing_core_dim='copy', # how to handle possibly missing core dims on input
)
ctd_interp = ctd_interp.drop_vars('z').assign_coords(
    {'z':('zlev',zlev, ctd_interp.z.attrs)}) # set z as 1D coord
ctd_interp

__d)__ Plot vertical cross sections of temperature, salinity and density with longitude and depth on the x- and y-axis, respectively.

__e)__ Calculate and plot the vertical shear of the meridional velocity, assuming the flow is in thermal wind balance.